In [35]:
pwd

'/home/dsc/Repositories'

In [2]:
cd ..

/home/dsc


In [3]:
cd ..

/home


In [21]:
!find -maxdepth 3 -type d -iname 'master*'

./dsc/Repositories/master-data-science


In [22]:
cd ./dsc/Repositories/master-data-science

/home/dsc/Repositories/master-data-science


In [23]:
#Este archivo tiene cambios. deberían visualizarse en la web github.

Hist_cmd/  practice/  README.md


In [1]:
!git add .


In [4]:
!git push


Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.12 KiB | 286.00 KiB/s, done.
Total 6 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/hguadalupe/master-data-science.git
   c8305bf..c5d59df  master -> master


In [36]:
cd ./master-data-science/practice/

/home/dsc/Repositories/master-data-science/practice


In [8]:
!find ../../../ -maxdepth 2 -iname 'data' 

../../../Data


In [37]:
arr_route = '../../../Data/challenge'

In [36]:
!find ../../../../Data/challenge -maxdepth 4 -type f -iname 'book*' 

../../../../Data/challenge/bookings.csv.bz2
../../../../Data/challenge/bookings.csv


In [16]:
!bzip2 -dk ../../../Data/challenge/bookings.csv.bz2


bzip2: Can't open input file ../../../Data/challenge!ls/bookings.csv.bz2: No such file or directory.


In [6]:
!ls ../../../../Data/challenge

bookings.csv  bookings.csv.bz2	searches.csv.bz2


In [10]:
!head ../../../../Data/challenge/bookings.csv

head: cannot open '2' for reading: No such file or directory
==> ../../../../Data/challenge/bookings.csv <==
act_date           ^source^pos_ctry^pos_iata^pos_oid  ^rloc          ^cre_date           ^duration^distance^dep_port^dep_city^dep_ctry^arr_port^arr_city^arr_ctry^lst_port^lst_city^lst_ctry^brd_port^brd_city^brd_ctry^off_port^off_city^off_ctry^mkt_port^mkt_city^mkt_ctry^intl^route          ^carrier^bkg_class^cab_class^brd_time           ^off_time           ^pax^year^month^oid      
2013-03-05 00:00:00^1A    ^DE      ^a68dd7ae953c8acfb187a1af2dcbe123^1a11ae49fcbf545fd2afc1a24d88d2b7^ea65900e72d71f4626378e2ebd298267^2013-02-22 00:00:00^1708^0^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHR     ^LON     ^GB      ^ZRH     ^ZRH     ^CH      ^LHRZRH  ^LONZRH  ^CHGB    ^1^LHRZRH         ^VI^T        ^Y        ^2013-03-07 08:50:00^2013-03-07 11:33:37^-1^2013^3^NULL     
2013-03-26 00:00:00^1A    ^US      ^e612b9eeeee6f17f42d9b0d3b79e75ca^7437560d8f276

# Aquí empieza el código

In [1]:
import pandas as pd
import numpy as np
import csv
from sqlalchemy import create_engine

## Tanteos python

In [6]:
#Ejercicio 1. Recuento de líneas (versión linux)
!wc -l ../../../../Data/challenge/bookings.csv 

^C


In [2]:
file = '../../../../Data/challenge/bookings.csv'


In [7]:
#Ejercicio 1. Tantea el archivo con cuidado, para ver contenido y tamaño.
pd_opening = pd.read_csv(file, sep='^',skip_blank_lines=True,nrows=10)
!ls -lah ../../../../Data/challenge/bookings.csv
print(list(pd_opening))
print(pd_opening.head())

type(pd_opening)

-rw-rw-r-- 1 dsc dsc 4.0G Jan  4  2016 ../../../../Data/challenge/bookings.csv
['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ', 'rloc          ', 'cre_date           ', 'duration', 'distance', 'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry', 'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry', 'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry', 'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class', 'brd_time           ', 'off_time           ', 'pax', 'year', 'month', 'oid      ']
   act_date             source  pos_ctry                          pos_iata  \
0  2013-03-05 00:00:00  1A      DE        a68dd7ae953c8acfb187a1af2dcbe123   
1  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
2  2013-03-26 00:00:00  1A      US        e612b9eeeee6f17f42d9b0d3b79e75ca   
3  2013-03-26 00:00:00  1A      AU        0f984b3bb6bd06661c95529bbd6193bc   
4  2013-03-26 00:00:00  1A      

pandas.core.frame.DataFrame

# Observaciones
Tenemos un archivo de 4Gigas y 10 millones de filas. VAmos a tener que dividirlo en partes. 
Podemos dividirlo en "chunks" de 100000 líneas con sqlalchemy, iterando sobre el CSV.

In [5]:
bookings_sample = pd.read_csv(file,sep='^',skip_blank_lines=True,nrows=1000)
bookings_sample.shape

(1000, 38)